# Exercise 03 : Aggregations

In [1]:
import pandas as pd
import sqlite3

## Create a connection to the database using the library sqlite3

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## Get the schema of the table test

In [3]:
pd.io.sql.read_sql("PRAGMA table_info(test);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## Get only the first 10 rows of the table test to check what the table looks like

In [4]:
query = "SELECT * FROM test LIMIT 10;"
pd.io.sql.read_sql(query, conn)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


## Find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query

- do this by joining the table with the table deadlines
- the difference should be displayed in hours
- do not take the lab ’project1’ into account, it has longer deadlines and will be an outlier
- the value should be stored in the dataframe df_min with the corresponding uid

In [5]:
pd.io.sql.read_sql("PRAGMA table_info(deadlines);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [6]:
query = "SELECT * FROM deadlines LIMIT 10;"
pd.io.sql.read_sql(query, conn)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


In [7]:
query = """
SELECT uid, MIN((deadlines.deadlines - strftime('%s', first_commit_ts)) / 3600) AS min_delta_hours
FROM test
JOIN deadlines ON test.labname = deadlines.labs
WHERE test.labname != 'project1'
GROUP BY uid;
"""
df_min = pd.io.sql.read_sql(query, conn)
df_min

,uid,min_delta_hours
0,user_1,6
1,user_10,39
2,user_14,84
3,user_17,34
4,user_18,3
5,user_19,32
6,user_21,33
7,user_25,2
8,user_28,8
9,user_3,60


## Do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [8]:
query = """
SELECT uid, MAX((deadlines.deadlines - strftime('%s', first_commit_ts)) / 3600) AS max_delta_hours
FROM test
JOIN deadlines ON test.labname = deadlines.labs
WHERE test.labname != 'project1'
GROUP BY uid;
"""
df_max = pd.io.sql.read_sql(query, conn)
df_max

,uid,max_delta_hours
0,user_1,175
1,user_10,132
2,user_14,200
3,user_17,81
4,user_18,10
5,user_19,148
6,user_21,126
7,user_25,150
8,user_28,174
9,user_3,182


## Do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg

In [9]:
query = """
SELECT AVG((deadlines.deadlines - strftime('%s', first_commit_ts)) / 3600) AS avg_delta_hours
FROM test
JOIN deadlines ON test.labname = deadlines.labs
WHERE test.labname != 'project1';
"""
df_avg = pd.io.sql.read_sql(query, conn)
df_avg

,avg_delta_hours
0,89.125


## We want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference

- using only one query, create a table with the columns: uid, avg_diff, pageviews
- uid is the uids that exist in the test
- avg_diff is the average delta between the first commit and the lab deadline per user
- pageviews is the number of Newsfeed visits per user
- do not take the lab ’project1’ into account
- store it to the dataframe views_diff
- use the Pandas method corr() to calculate the correlation coefficient between the number of pageviews and the difference

In [10]:
query = """
SELECT test.uid, 
       AVG((deadlines.deadlines - strftime('%s', test.first_commit_ts)) / 3600) AS avg_diff, 
       COUNT(pageviews.uid) AS pageviews
FROM test
LEFT JOIN pageviews ON test.uid = pageviews.uid
JOIN deadlines ON test.labname = deadlines.labs
WHERE test.labname != 'project1'
GROUP BY test.uid;
"""
views_diff = pd.io.sql.read_sql(query, conn)
correlation_coefficient = views_diff['pageviews'].corr(views_diff['avg_diff'])
print("\nCorrelation coefficient between the number of pageviews and the difference:")
print(correlation_coefficient)


Correlation coefficient between the number of pageviews and the difference:
0.18583402677246152


## Close the connection

In [11]:
conn.close()